[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aicrumb/CQKP/blob/master/CQKP_Example.ipynb)

In [1]:
!pip install git+https://github.com/aicrumb/CQKP
from google.colab import output
output.clear() # we just do this so the notebook looks cleaner

In [2]:
import cqkp
model = cqkp.load_model()
output.clear()  # same here, just cleaning up

In [3]:
question = "what is a cow?"
answers = [
    "Cattle are large domesticated bovines. They are most widespread species of the genus Bos. ",
    "Cows eat grass and hay. On a dry matter basis a cow will eat 4 to 5 tons of forage per year."
]
print(model.best_answer(question, answers)[0])

Cattle are large domesticated bovines. They are most widespread species of the genus Bos. 


In [4]:
answer = "Cattle are large domesticated bovines. They are most widespread species of the genus Bos."
questions = [
    "what are pigs",
    "what are cows"
]
print(model.best_question(questions, answer)[0])

what are cows


In [5]:
# to get just encodings, it requires both to be met though
questions = model.tokenize(["what is a duck"])
answers = model.tokenize([""])

# index zero is the questions, index 1 is the answers
encoding = model.score(questions,answers)[0]

# the shape of the encoding, showing it's a 1024 dim vector
print(encoding.shape)

# if you just want to use this as a universal sentence encoder, i'd use the answers part of the model as it's built for long form text

torch.Size([1, 1024])
